In [1]:
import pandas as pd
import numpy as np
import os
from collections import defaultdict
import re

import gensim
import os
import collections
import smart_open
import random
import pickle

#Get a list of the CSV files 
with open(r"train_corpus.pickle", "rb") as input_file:
        train_corpus = pickle.load(input_file)


In [5]:
def formatting(train_corpus):
    sentence_list = [(train_corpus[i].words, train_corpus[i].tags[0]) for i in range(len(train_corpus))]
    lst = [(sentence, 1) if i<193 else (sentence, 0) for (sentence, i) in sentence_list ]
    lst = [(sentence,i) for (sentence, i) in lst  if sentence]
    return lst

In [7]:
def guess_candidate(train_corpus):
    ## some formatting for the data before we start
    lst = formatting(train_corpus)
        
    ## download data from previous attempts
    ## if this is the first time then create and empty dict
    try:
        with open(r"candidate_guess.pickle", "rb") as input_file:
            d = pickle.load(input_file)
    except:
        d = {}

    ## if user already attempted get the record
    name = input('please enter your name').lower()
    name = 'murat'
    if name in d.keys():
        get_previous_data = d[name]
        num_correct = get_previous_data['num_correct']
        old_sample = get_previous_data['sample_size']
        old_sentences = get_previous_data['guessed_sentences']
        old_score = get_previous_data['score']
        ## delete previously attempted sentences from list
        lst = [(s, i) for (s, i) in lst if (s, i) not in old_sentences]
    else:
        d[name] = {'sample_size':0, 
                   'guessed_sentences': [],
                   'score': 0,
                   'attempted':0,
                   'num_correct':0
                  }
        get_previous_data = d[name]
        num_correct = get_previous_data['num_correct']
        old_sample = get_previous_data['sample_size']
        old_sentences = get_previous_data['guessed_sentences']
        old_score = get_previous_data['score']
    
    ## ask user for how many sentence they want to label
    sample_size = int(input('How many senteces do you want to guess? (0, 1000)\n'))
    
    ## adjust the record
    d[name]['attempted'] += sample_size
    random_list = np.random.randint(0, len(lst), sample_size)
    
    ## create random sentences 
    sample_list = [lst[i] for i in random_list ]
    ## adjust the record
    d[name]['guessed_sentences'] += sample_list
    
    ## set correct answers to zero for this game
    correct = 0
    for sentence in sample_list:
        input('press any key to continue...\n')
        guess = input('{} \nBernie or Trump? Enter 1 for Bernie and 0 for Trump\n'.format(sentence[0]))
        tag = sentence[1]
        if int(guess)==tag:
            print('congrulations!!')
            correct +=1
        else:
            print('The correct answer was {}'.format(tag))
    d[name]['score'] = (num_correct+correct)/(d[name]['attempted'])
    d[name]['num_correct'] += correct
    print('score %.3f \noverall human score is %.3f'%(correct/sample_size, d[name]['score']))
    with open(r"candidate_guess.pickle", "wb") as output_file:
        pickle.dump(d, output_file)
            
        

In [8]:
guess_candidate(train_corpus)

please enter your namemurat
How many senteces do you want to guess? (0, 1000)
1
press any key to continue...

['had', 'artificial', 'obstacles', 'put', 'in', 'our', 'path'] 
Bernie or Trump? Enter 1 for Bernie and 0 for Trump
1
The correct answer was 0
score 0.000 
overall human score is 0.000
